# Twitter Scrapper
The crawling process was done using Tweet-Harvest. Written by Helmi Satria on  March 30th 2024.



In [5]:
import os

In [18]:
# #@title Twitter Auth Token

# twitter_auth_token = '####'

In [6]:
!pip install pandas

!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

!NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

!sudo apt-get update
!sudo apt-get install nodejs -y

!node -v


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Command not found
Command not found
'NODE_MAJOR' is not recognized as an internal or external command,
operable program or batch file.
Command not found
Command not found


v22.9.0


SCRAPING WITH TOKEN FALLBACK

In [ ]:
import subprocess
import time
import pandas as pd

twitter_auth_tokens = ['###', '###']  
filename = 'fomo.csv'
search_keyword = (
    '"takut ketinggalan" OR "biar kekinian" OR "harus ikut tren" OR '
    '"anak FOMO banget" OR "gue doang yang belum" OR "fomo konser" OR '
    '"fomo tiktok" OR "fomo skincare" OR "ikut-ikutan tren" OR '
    '"demi konten" OR "ikut tren tiktok" since:2025-02-01 until:2025-04-25 lang:id'
)
limit = 10000

def run_command_with_token(token, output_file):
    command = [
        "npx", "-y", "tweet-harvest@2.6.1",
        "-o", output_file,
        "-s", search_keyword,
        "--tab", "LATEST",
        "-l", str(limit),
        "--token", token
    ]

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    for line in process.stdout:
        print(line, end="")

    process.wait()
    return process.returncode

def remove_duplicates(files):
    df_list = [pd.read_csv(file) for file in files]
    combined_df = pd.concat(df_list, ignore_index=True)

    combined_df.drop_duplicates(subset=['tweet_id'], keep='first', inplace=True)
    combined_df.to_csv('fomo_no_duplicates.csv', index=False)
    print("Data telah digabungkan dan duplikat dihapus.")

output_files = []
for token in twitter_auth_tokens:
    print(f"Trying with token: {token}")
    temp_file = f"fomo_{token}.csv"
    result = run_command_with_token(token, temp_file)

    if result == 0:
        print(f"Successfully fetched data with token: {token}")
        output_files.append(temp_file)
        break
    else:
        print(f"Rate limit exceeded with token: {token}. Retrying with next token...")
        time.sleep(15 * 60) 

if output_files:
    remove_duplicates(output_files)


In [20]:
import pandas as pd
file_path = f"tweets-data/{filename}"
df = pd.read_csv(file_path, delimiter=",")
display(df)

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1915461760418799957,Thu Apr 24 17:44:05 +0000 2025,1,suara jaehyun tuh tipe suara yg pengen km deng...,1915461760418799957,NaN,NaN,in,cisey brainrot,0,1,0,https://x.com/withcheesejae/status/19154617604...,1356203486728380416,withcheesejae
1,1915436413882864068,Thu Apr 24 16:08:49 +0000 2025,0,@catchingdoie yaampun kak aku auto liat jam ta...,1915437786108449070,NaN,catchingdoie,in,NaN,0,1,0,https://x.com/ddoykuu/status/1915437786108449070,1578447234261078016,ddoykuu
2,1915437708966846752,Thu Apr 24 16:08:31 +0000 2025,0,Td pagi lari2 di tangga stasiun mrt fatmawati ...,1915437708966846752,NaN,NaN,in,NaN,0,0,0,https://x.com/gilangnw/status/1915437708966846752,101781442,gilangnw
3,1915434996837605794,Thu Apr 24 15:57:44 +0000 2025,0,mau bobo tp takut ketinggalan sesuatu nanti,1915434996837605794,NaN,NaN,in,NaN,0,0,0,https://x.com/ffnyacA/status/1915434996837605794,1753344810197659649,ffnyacA
4,1915430452011426253,Thu Apr 24 15:39:41 +0000 2025,0,okey siap beberapa hari kedepan ga boleh tidur...,1915430452011426253,NaN,NaN,in,NaN,0,0,0,https://x.com/lilhosun/status/1915430452011426253,1845630628978675712,lilhosun
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,1911317496709386632,Sun Apr 13 07:18:34 +0000 2025,0,@EmhadyZ Smartphone yang sulit dilepaskan buka...,1911318075401712104,NaN,EmhadyZ,in,wherever you are,0,0,0,https://x.com/grok/status/1911318075401712104,1720665183188922368,grok
534,1911294554709672182,Sun Apr 13 06:42:55 +0000 2025,0,@Zaxveline Emang pada makin bege demi konten,1911309104263631220,NaN,Zaxveline,in,NaN,0,0,0,https://x.com/Yuureikunkun/status/191130910426...,2545400612,Yuureikunkun
535,1911033653557436417,Sun Apr 13 06:41:03 +0000 2025,0,@itstimehan @txtfromkeuangan Halo! Emas itu ka...,1911308635680227615,https://pbs.twimg.com/media/GnglrYbbAAAbx1A.jpg,itstimehan,in,wherever you are,0,0,0,https://x.com/grok/status/1911308635680227615,1720665183188922368,grok
536,1911241729837351338,Sun Apr 13 06:36:39 +0000 2025,0,@expenxsive Gak usah takut ketinggalan yq,1911307525993251006,NaN,expenxsive,in,NaN,0,0,0,https://x.com/hierasyaaa/status/19113075259932...,1312429196249952256,hierasyaaa


In [5]:
# Cek jumlah data yang didapatkan

num_tweets = len(df)
print(f"Jumlah tweet dalam dataframe adalah {num_tweets}.")

Jumlah tweet dalam dataframe adalah 108.
